In [2]:
from tensor import TensorSpec, Tensor, TensorShape
from utils.index import Index
from collections.dict import Dict, KeyElement
from random import rand

In [3]:
@value
struct IntKey(KeyElement):
    var n: Int

    fn __init__(inout self, owned n: Int):
        self.n = n

    fn __hash__(self) -> Int:
        return hash(self.n)

    fn __eq__(self, other: Self) -> Bool:
        return self.n == other.n


let fashion_mnist_key = Dict[IntKey, String]()
fashion_mnist_key[0] = "T-shirt/top"
fashion_mnist_key[1] = "Trouser"
fashion_mnist_key[2] = "Pullover"
fashion_mnist_key[3] = "Dress"
fashion_mnist_key[4] = "Coat"
fashion_mnist_key[5] = "Sandal"
fashion_mnist_key[6] = "Shirt"
fashion_mnist_key[7] = "Sneaker"
fashion_mnist_key[8] = "Bag"
fashion_mnist_key[9] = "Ankle boot"


In [4]:
fn uint8_to_int32(uint8_tensor: Tensor[DType.uint8]) raises -> SIMD[DType.int32, 1]:
    let result = (
        (uint8_tensor[0].to_int() << 24)
        | (uint8_tensor[1].to_int() << 16)
        | (uint8_tensor[2].to_int() << 8)
        | uint8_tensor[3].to_int()
    )
    return SIMD[DType.int32, 1](result)


In [5]:
fn get_slice[type: DType](tensor: Tensor[type], start_index: Int, end_index: Int) raises -> Tensor[type]:
    if end_index < start_index:
        raise "End index more than start index."
    elif end_index == start_index:
        var output_tensor = Tensor[type](1)
        output_tensor[0] = tensor[start_index]

        return output_tensor
    else:
        var output_tensor = Tensor[type](end_index - start_index)
        for i in range(start_index, end_index):
            output_tensor[i - start_index] = tensor[i]
        return output_tensor


In [6]:
fn read_data_as_images(images_path: Path) raises -> Tensor[DType.uint8]:
    if images_path.exists():
        let image_file = images_path.read_bytes().astype[DType.uint8]()
        let num_images = uint8_to_int32(get_slice[DType.uint8](image_file, 4, 8))
        let width = uint8_to_int32(get_slice[DType.uint8](image_file, 8, 12))
        let height = uint8_to_int32(get_slice[DType.uint8](image_file, 12, 16))

        var image_data = get_slice(image_file, 16, image_file.num_elements())

        let images_shape = TensorShape(
            num_images.to_int(), height.to_int(), width.to_int()
        )

        let images = image_data.reshape(images_shape)
        return images
    raise "The images directory does not exist."


In [7]:
fn read_data_as_labels(labels_path: Path) raises -> Tensor[DType.uint8]:
    if labels_path.exists():
        let label_file = labels_path.read_bytes().astype[DType.uint8]()
        let num_labels = uint8_to_int32(get_slice[DType.uint8](label_file, 4, 8))

        let labels = get_slice(label_file, 8, num_labels.to_int() + 8)

        return labels
    raise "The labels directory does not exist."


In [8]:
fn tensor_print[type: DType](index: Int, tensor: Tensor[type]):
    var cur_line: String

    fn get_char_for_pixel(pixel_value: Int) -> String:
        if pixel_value == 0:
            return " "
        elif pixel_value < 32:
            return "."
        elif pixel_value < 64:
            return ","
        elif pixel_value < 96:
            return ":"
        elif pixel_value < 128:
            return ";"
        elif pixel_value < 160:
            return "o"
        elif pixel_value < 192:
            return "O"
        elif pixel_value < 224:
            return "X"
        else:
            return "#"

    for j in range(tensor.shape()[1]):
        cur_line = ""
        for k in range(tensor.shape()[2]):
            cur_line += get_char_for_pixel(tensor[Index(index, j, k)].to_int()) + " "
        print(cur_line)


In [9]:
let base_dir = "/Users/tprazak/Documents/seminary_work_nn/MNIST_in_mojo/"
let image_path_mnist = base_dir + "mnist/train-images.idx3-ubyte"
let labels_path_mnist = base_dir + "mnist/train-labels.idx1-ubyte"
let image_path_fashion = base_dir + "fashion_mnist/train-images-idx3-ubyte"
let labels_path_fashion = base_dir + "fashion_mnist/train-labels-idx1-ubyte"
let images = read_data_as_images(image_path_mnist)
let labels = read_data_as_labels(labels_path_mnist)
let fashion_images = read_data_as_images(image_path_fashion)
let fashion_labels = read_data_as_labels(labels_path_fashion)


In [10]:
fn matmul[type: DType](first_matrix: Tensor[type], second_matrix: Tensor[type]) raises -> Tensor[DType.float32]:
    let f_m = first_matrix.astype[DType.float32]()
    let s_m = second_matrix.astype[DType.float32]()
    if first_matrix.rank() != 2 or second_matrix.rank() != 2:
        raise 'At least one of the tensors is not a matrix'
    if first_matrix.dim(1) != second_matrix.dim(0):
        raise 'Then matrices are not compatible for matrix multiplication'

    let o_m_rows = f_m.dim(0)
    let o_m_columns = s_m.dim(1)
    var o_m = Tensor[DType.float32](o_m_rows * o_m_columns)
    for i in range(o_m_rows):
        for j in range(f_m.dim(1)):
            for k in range(o_m_columns):
                o_m[i*o_m_columns+k] += f_m[i, j] * s_m[j, k]
    return o_m.reshape(TensorShape(o_m_rows, o_m_columns))

In [11]:
alias nelts = simdwidthof[DType.float32]()
fn matmul_vectorized[type: DType](first_matrix: Tensor[type], second_matrix: Tensor[type]) raises -> Tensor[DType.float32]:
    let f_m = first_matrix.astype[DType.float32]()
    let s_m = second_matrix.astype[DType.float32]()
    if first_matrix.rank() != 2 or second_matrix.rank() != 2:
        raise 'At least one of the tensors is not a matrix'
    if first_matrix.dim(1) != second_matrix.dim(0):
        raise 'Then matrices are not compatible for matrix multiplication'

    let o_m_rows = f_m.dim(0)
    let o_m_cols = s_m.dim(1)
    var o_m = Tensor[DType.float32](o_m_rows * o_m_cols)
    for i in range(o_m_rows):
        for j in range(f_m.dim(1)):
            for kv in range(0, nelts*(o_m_cols//nelts), nelts):
                o_m.simd_store[nelts](i*o_m_cols+kv, o_m.simd_load[nelts](i*o_m_cols+kv) + f_m.simd_load[1](i*f_m.dim(1)+j) * s_m.simd_load[nelts](j*s_m.dim(1)+kv))
            for k in range(nelts*(o_m_cols//nelts), o_m_cols):
                o_m.simd_store[1](i*o_m_cols+k, o_m.simd_load[1](i*o_m_cols+k) + f_m.simd_load[1](i*f_m.dim(1)+j) * s_m.simd_load[1](j*s_m.dim(1)+k))
    return o_m.reshape(TensorShape(o_m_rows, o_m_cols))

In [22]:
struct LayerDense:
    var input: Tensor[DType.float32]
    var output: Tensor[DType.float32]
    var weights: Tensor[DType.float32]
    var biases: Tensor[DType.float32]

    fn __init__(inout self, num_inputs: Int, num_neurons: Int):
        self.weights = 0.01 * rand[DType.float32](num_inputs, num_neurons)
        self.biases = Tensor[DType.float32](1, num_neurons)
        self.input = Tensor[DType.float32]()
        self.output = Tensor[DType.float32]()
    
    fn forward(inout self, inputs: Tensor[DType.float32]) raises:
        self.output = matmul_vectorized(inputs, self.weights)
        self.input = inputs

    
        

In [25]:
var inputs = rand[DType.float32](12)
print(inputs)
inputs = inputs.reshape(TensorShape(4, 3))
print(inputs)
var d1 = LayerDense(3, 5)
d1.forward(inputs)
print(d1.output)


Tensor([[0.063639961183071136, 0.22221310436725616, 0.61379098892211914, ..., 0.94909024238586426, 0.17877237498760223, 0.83520811796188354]], dtype=float32, shape=12)
Tensor([[0.063639961183071136, 0.22221310436725616, 0.61379098892211914],
[0.91744643449783325, 0.23601752519607544, 0.43634861707687378],
[0.61380642652511597, 0.84160757064819336, 0.55225920677185059],
[0.94909024238586426, 0.17877237498760223, 0.83520811796188354]], dtype=float32, shape=4x3)
Tensor([[0.0072466549463570118, 0.0079225338995456696, 0.0063172997906804085, 0.0016481641214340925, 0.0047957035712897778],
[0.0096768364310264587, 0.012567325495183468, 0.0063163670711219311, 0.0044222073629498482, 0.0039474521763622761],
[0.01461403351277113, 0.017364505678415298, 0.0072549376636743546, 0.0042622163891792297, 0.0064770951867103577],
[0.012569409795105457, 0.015667194500565529, 0.010263418778777122, 0.0052178371697664261, 0.0064408816397190094]], dtype=float32, shape=4x5)
